# Twisted Probabilities, Uncerntainty, and Prices
by [Lars Peter Hansen](https://larspeterhansen.org/), [Bálint Szőke](https://www.balintszoke.com/), Lloyd S. Han and [Thomas J.Sargent](http://www.tomsargent.com/), you could find the latest draft [here](https://larspeterhansen.org/research/papers/). 

We illustrate effects of concerns about robustness in three environments using a model of capital accumulation with adjustment costs proposed by [Eberly & Wang (2011)](https://www0.gsb.columbia.edu/faculty/nwang/papers/EberlyWang2011_04v3.pdf). We modify their model to expose investment returns  to long-run risks and make investors concerned  about misspecifications of them. Three distinct example economies environments feature:
* a single capital stock
* two capital stocks with a common exposure to long-run risk
*  two capital stocks with only one being exposed to long-run risk

The notebook presents the code for single capital model accompanying section 6 in the paper

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Baseline-model" data-toc-modified-id="Baseline-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Baseline model</a></span><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Calibration" data-toc-modified-id="Calibration-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calibration</a></span></li></ul></li><li><span><a href="#Single-Capital-Stock-Case" data-toc-modified-id="Single-Capital-Stock-Case-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Single Capital Stock Case</a></span><ul class="toc-item"><li><span><a href="#Parameter-alternatives" data-toc-modified-id="Parameter-alternatives-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Parameter alternatives</a></span></li><li><span><a href="#Planner's-Problem" data-toc-modified-id="Planner's-Problem-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Planner's Problem</a></span></li></ul></li></ul></div>

In [5]:
# Install and import Necessary packages
using Pkg
Pkg.add("Optim")
Pkg.add("Roots")
Pkg.add("NPZ")
Pkg.add("Interpolations")
using Optim
using Roots
using NPZ

include("newsets_utils.jl")

 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


shock_price (generic function with 1 method)

## Baseline model

### Setup

Aggregate output is proportional to a single capital stock with a constant productivity parameter $\mathcal{A}$. A representative household cares about consumption $C$ with instantaneous utility $\delta\log C$. Under the baseline model, investment $I$ affects capital $K$ according to

\begin{align}
dK_t &= \left[\frac{I_t}{K_t} - \frac{\phi}{2}\left(\frac{I_t}{K_t}\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right)\right]K_t dt + (.01)\sigma_k K_t\cdot dW_t \label{eqn:model}\\
dZ_t &= \left(\widehat\alpha_z -\widehat \kappa Z_t \right)dt + \sigma_z \cdot dW_t \nonumber
\tag{1}
\end{align}

with adjustment cost parameter $\phi$. With zero investment, the rate of change of capital is $(.01)\left(\widehat\alpha_k + \widehat\beta Z_t\right)$, where $Z$ is an exogenously specified continuous-time scalar autoregressive process  that puts long-term risks into  returns on capital. Under baseline model $(1)$, the mean of $Z$ is  ${\overline z} = {\frac {{\widehat \alpha}_z} {{\widehat \kappa}}}$.

he aggregate resource constraint is $C + I = {\cal A} K$. It is convenient to pose a robust planner's problem in terms of an investment-capital ratio $D_t \doteq\frac{I_t}{K_t}$ and $\log K_t$ that satisfy
\begin{align*}
d \log K_t = \left[D_t - \frac{\phi}{2}\left(D_t\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right) - {\frac {(.01)^2 |\sigma_k|^2}{2}} \right] dt + (.01)\sigma_k \cdot dW_t.
\end{align*}
Denote state variable as $X_t = [\log K_t,\ Z_t]'$ and instantaneous utility function
\begin{align*}
\upsilon(X_t, D_t) = \delta\log\left({\mathcal A}- D_t\right) + \delta\log K_t.
\end{align*}



### Calibration

The basis of our quantitative analysis is an empirical model of  aggregate consumption dynamics. Our code for VAR estimation could be found in __VAR Estimation__ section of Git hub repository [Tenuous Belief](https://github.com/lphansen/TenuousBeliefs).

We follow [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf) by fitting a trivariate VAR for macroeconomic time series that contain information about long-term consumption growth, namely,
log consumption growth, the difference between logs of business income and consumption, and the difference between logs of personal dividend income and
consumption.

The time series are quarterly data from 1948 Q1 to 2018 Q2. Our consumption measure is per capita consumption of non-durables and services from NIPA Table 1.1.5. Business income is measured as proprietor's income plus corporate profits per capita and the series are from NIPA Table 1.12. Personal dividend income is from NIPA Table 2.1. By including proprietors' income in addition to corporate profits, we use a broader measure of business income than [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf), who used only corporate profits. Moreover, [Hansen et al. (2008)](http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf) did not include  personal dividends in their VAR analysis.
We restrict  all three time series to have a common martingale component by imposing a known cointegration relation among them.

We convert the discrete time VAR estimates to baseline parameters $({\widehat \alpha}_k,  {\widehat \beta}, {\widehat \alpha}_z, {\widehat \kappa} )$ and $(\sigma_c, \sigma_z)$ by setting ${\widehat \alpha}_z=0$ and $\widehat{\beta}=1$ and matching the dynamics of the VAR implied long-term consumption growth forecast with those of $Z$.
As a result, we obtain the following parameters for the baseline model of consumption $(1)$:

\begin{align} 
& \begin{matrix}
{\widehat \alpha}_c  & = & .484 & & {\widehat \beta} &= &1 \cr
{\widehat \alpha}_z  &= & 0 &  & {\widehat \kappa} & = & .014 \end{matrix} \cr
& \sigma = \begin{bmatrix}
(\sigma_c)' \cr (\sigma_z)' \end{bmatrix}  =  \begin{bmatrix} .477 & 0 \cr  .011 & .025 \end{bmatrix} .
\end{align}

We set the household's subjective discount rate equal to $\delta=.002.$

In more detail, we choose $\widehat{\alpha}_c$ and $(1-\widehat{\kappa})$ to match the VAR implied unconditional mean of consumption growth and autoregressive coefficient of expected long-term consumption growth, respectively. In addition, we set $(\sigma_c, \sigma_z)$ such that $(1, \widehat{\kappa}^{-1})\sigma\sigma'(1, \widehat{\kappa}^{-1})'$ is equal to the VAR implied covariance matrix of the one-step-ahead forecast error for consumption growth and expected long-term consumption growth. We achieve identification by imposing a lower triangular structure with positive diagonal elements on $\sigma$.

In [6]:
#==============================================================================#
#    PARAMETERS
#==============================================================================#
delta = 0.002;

# (1) Baseline model
alpha_z_hat = 0.0;
kappa_hat = 0.014;

alpha_c_hat = 0.484;       # consumption intercept (estimated) 
beta_hat = 1.0;
sigma_c = [0.477, 0.0  ];   # consumption exposure (= exposure of single capital)
sigma_z = [0.011, 0.025];

## Single Capital Stock Case

### Parameter alternatives

A planner is concerned about  alternative models for  capital evolution than baseline model $(1)$ in the following parametric class

\begin{align}
d \log K_t &= \left[D_t - \frac{\phi}{2}\left(D_t\right)^2 + (.01)\left( \alpha_k +  \beta Z_t \right) - {\frac {(.01)^2 |\sigma_k|^2}2 } \right] dt + (.01)\sigma_k \cdot dW^H_t \label{eqn:model2} \\
dZ_t &= \left(\alpha_z - \kappa Z_t \right)dt + \sigma_z \cdot dW^H_t \nonumber
\tag{2}
\end{align}

where $W^H$ denotes a distorted shock process of the form:

\begin{equation}
dW_t = H_t dt + d{W}_t^H,
\end{equation}

Uncertainty about $(\alpha_k,\beta)$ describes unknown components in  returns to investment while uncertainty about $(\alpha_z,\kappa)$ captures an unknown growth evolution process. We represent a model of the form $(2)$ by restricting the drift distortion $H$ for $W$ to satisfy

\begin{equation}\label{eq:affine_drift}
H_t = \eta(X_t) \doteq {\eta }_0  + {\eta }_1 Z_t
\end{equation}

and using $(1)$  and  $(2)$ to deduce the following restrictions on $\eta_0$ and $\eta_1$ as functions of $(\alpha_k, \beta, \alpha_z,  \kappa)$:
\begin{align}
\sigma {\eta}_0  = \begin{bmatrix} {\alpha}_k - {\widehat \alpha}_k \cr {\alpha}_z  -{\widehat \alpha}_z \end{bmatrix}, \hspace{2cm}
\sigma {\eta}_1  = \begin{bmatrix} \beta - {\widehat \beta}  \cr {\widehat \kappa} - { \kappa}  \end{bmatrix}
\tag{3}
\end{align}
where
\begin{align}
\sigma \doteq \begin{bmatrix} (\sigma_k)' \cr (\sigma_z)' \end{bmatrix} .
\end{align}

Pairs $(\eta_0, \eta_1)$ that satisfy restrictions $(3)$ represent models having parametric form $(2)$.

### Planner's Problem

To capture a desire for robustness to parametric misspecifications represented by the class $(2)$, we suppose that the twisting function has the form $(4)$ and seek a function $V(z, k; \ell) = \log k + \nu(z; \ell)$ that satisfies the robust planner's HJB equation
\begin{align}\label{hjb:robust}
0 = & \max_{d} \min_{h} \ \ - \delta \nu(z; \ell) + \delta \log ({\mathcal A} - d) + \left[d - {\frac {\phi} 2} d^2 +  (.01) \left({\widehat \alpha}_k + {\widehat \beta} z - \frac{(.01)|\sigma_k^2|}{2}\right)\right] \cr
& \quad\quad + \frac{\partial \nu(z; \ell)}{ \partial z}\left( {\widehat \alpha}_z -{\widehat \kappa} z \right) + \frac{1}{2}\frac{ \partial^2 \nu(z; \ell)}{\partial z^2} |\sigma_z|^2 \nonumber \\
& \quad\quad + (.01) \left[\sigma_k \cdot h\right] + \frac{\partial \nu(z; \ell)}{\partial z}\left[\sigma_z \cdot h\right]  +  \frac{\ell}{2}\left[ |h|^2 - \xi(z) \right].
\tag{5}
\end{align}

The last line of $(5)$ depicts how concerns about robustness enter the HJB equation via $h$. The minimizing choice of $h$ depends on the state $z$ through the marginal value function:

\begin{align}
H^*(z; \ell) & \doteq \begin{bmatrix}h^*_k \\ h^*_z \end{bmatrix} = -\frac{1}{\ell}\left[ (.01) \sigma_k + \frac{\partial  \nu(z ; \ell)}{\partial z}\sigma_z  \right]. \tag{6}
\end{align}

The  maximizing choice of $d$ solves:

\begin{align}\label{eqn:d_star}
 1 - \phi d^* =   {\frac \delta  {{\mathcal A} - d^*}}
\end{align}

where $d^*$ denotes the chosen investment-capital ratio. Notice that  $d^*$ is constant and does not depend on the underlying state variables, so the optimal consumption-capital ratio $c^*=\mathcal{A}-d^*$ is also constant. Because neither $c^*$ nor $d^*$ is affected by
robustness concerns,  under the baseline model capital and  consumption dynamics are not altered by robustness concerns. We write the implied dynamics for the logarithm of consumption as:

\begin{align}
d \log C_t  = (.01)\left({\widehat \alpha}_c  +  {\widehat \beta} Z_t\right) dt + (.01)\sigma_c \cdot dW_t \label{eq:baseline_consumption}
\end{align}

where

\begin{align}\label{eq:alpha_c}
  {\widehat \alpha_c} = 100\left(d^* - \frac{\phi}{2}\left(d^*\right)^2\right) +  {\widehat \alpha}_k  - {\frac {(.01) |\sigma_k|^2}2 } \hspace{15mm}\text{and}\hspace{15mm} \sigma_c = \sigma_k.
\end{align}

In Appendix C.1, we show that the value function $\nu(z; \ell)$ is quadratic in $z$. As a consequence, $h^*$ is affine in $z$. The multiplier $\ell^*$ is then determined by solving

\begin{align*}
\ell^*(z_0) = \arg\max_{\ell>0} \ \nu(z_0; \ell)
\end{align*}

where $z_0$ is an initial value of $Z$. We can find the worst-case model by substituting $\ell^*(z_0)$ in equation $(6)$.

For codes solving planner's problem in single stock capital model, please refer to __newsets_singlecapital.jl__ under __code__ folder

In [ ]:
#===========================  CALIBRATION  ====================================#
# consumption_investment = 3.1;
# A, phi, alpha_k_hat, investment_capital = calibration2(40.,
#                                             consumption_investment,
#                                             alpha_c_hat, delta, sigma_c)


A = 0.05
phi = 28
investment_capital, consumption_investment, alpha_k_hat = calibration3(phi,
                                            A, delta, alpha_c_hat, sigma_c)

println("  Calibrated values: A:", A,
        "  phi: ", phi,
        "  alpha_k : ", alpha_k_hat,
        "  C/I : ", consumption_investment,
        "  I/K : ", investment_capital)
println("=============================================================")
#==============================================================================#
baseline = Baseline(alpha_z_hat, kappa_hat, sigma_z,
                    alpha_c_hat, beta_hat, sigma_c, delta);
technology = Technology(A, phi);

# optimal investment-capital ratio:
d_star = dstar_singlecapital(technology, baseline);

# single capital case should give rise to observed consumption growth:
alpha_c2 = 100*(d_star-phi*d_star^2/2)+alpha_k_hat-(0.01)*dot(sigma_c,sigma_c)/2;

println("alpha_c_hat =", baseline.alpha_k_hat,
        " technology parameters imply ", alpha_c2, '\n');


#=================== Table 1&2 ================================================#
kappa_list = [0.01, 0.005];
kappa_list2 = [0.014, 0.01, kappa_tilde];
q_list = [0.1, 0.2];

store_matrix = zeros(length(kappa_list)+length(kappa_list2)*length(q_list), 25);

for ki=1:length(kappa_list)
    H0, H1, ak, b, az, k, dc, re, q, l_star = worst_case(alpha_z_hat,
                                                            kappa_list[ki],
                                                            baseline.alpha_k_hat,
                                                            baseline.beta_hat,
                                                            baseline, technology);
    store_matrix[ki, 1:4] = [q, alpha_z_hat, kappa_list[ki], baseline.beta_hat]
    store_matrix[ki, 5:11] = [ak, b, az, k, dc, re, q];
    res = optimize(r -> chernoff_objective_single(r, H0, H1, baseline), 0.0, 1.0);
    chernoff = - Optim.minimum(res)
    if chernoff == 0.0
        halflife = Inf
    else
        halflife = log(2.0) / chernoff
    end
    store_matrix[ki, 12:13] = [chernoff, halflife];
    q, re, c, hl = worrisome_entropy(alpha_z_hat, kappa_list[ki],
                                        baseline.alpha_k_hat, baseline.beta_hat, baseline);
    store_matrix[ki, 14:17] = [q, re, c, hl]
    store_matrix[ki, 18:19] = H0'
    store_matrix[ki, 20:21] = H1'
    store_matrix[ki, 22] = l_star
    x0, x1, x2 = tilting_function(Worrisome(alpha_z_hat,
                                            kappa_list[ki],
                                            baseline.alpha_k_hat,
                                            baseline.beta_hat), baseline)
    store_matrix[ki, 23:25] = [x0, x1, x2]
end

for ki=1:length(kappa_list2)
    for qi=1:length(q_list)
        alpha_tilde, wc = choose_alpha(q_list[qi],kappa_list2[ki],
                                baseline.alpha_k_hat,baseline.beta_hat,baseline,technology);
        H0, H1, ak, b, az, k, dc, re, q, l_star = wc;

        row_index = length(kappa_list) + (ki-1)*length(q_list) + qi
        store_matrix[row_index, 1:4] = [q, alpha_tilde, kappa_list2[ki], baseline.beta_hat];
        store_matrix[row_index, 5:11] = [ak, b, az, k, dc, re, q];
        res = optimize(r -> chernoff_objective_single(r,H0,H1,baseline), 0.0, 1.0);
        chernoff = - Optim.minimum(res)
        if chernoff == 0.0
            halflife = Inf
        else
            halflife = log(2.0) / chernoff
        end
        store_matrix[row_index, 12:13] = [chernoff, halflife];
        q, re, c, hl = worrisome_entropy(alpha_tilde, kappa_list2[ki],
                                        baseline.alpha_k_hat, baseline.beta_hat, baseline);
        store_matrix[row_index, 14:17] = [q, re, c, hl]
        store_matrix[row_index, 18:19] = H0'
        store_matrix[row_index, 20:21] = H1'
        store_matrix[row_index, 22] = l_star
        x0, x1, x2 = tilting_function(Worrisome(alpha_tilde,
                                                kappa_list2[ki],
                                                baseline.alpha_k_hat,
                                                baseline.beta_hat), baseline)
        store_matrix[row_index, 23:25] = [x0, x1, x2]
    end
end

store_matrix2 = zeros(length(kappa_list2[2:end])*length(q_list), 25);

for ki=1:length(kappa_list2[2:end])
    for qi=1:length(q_list)
        row_index = (ki-1)*length(q_list) + qi

        # alpha_z_tilde = minimum_xi(store_matrix[length(kappa_list) + row_index , 2],
        #                            alpha_z_hat, sigma_z)
        alpha_z_tilde = store_matrix[length(kappa_list) + 2 + row_index , 2]
        beta_tilde, wc = choose_beta(q_list[qi], alpha_z_tilde, baseline.kappa_hat,
                                    baseline.alpha_k_hat, baseline, technology);
        H0, H1, ak, b, az, k, dc, re, q, l_star = wc;

        store_matrix2[row_index, 1:4] = [q, alpha_z_tilde, baseline.kappa_hat, beta_tilde];
        store_matrix2[row_index, 5:11] = [ak, b, az, k, dc, re, q];
        res = optimize(r -> chernoff_objective_single(r,H0,H1,baseline), 0.0, 1.0);
        chernoff = - Optim.minimum(res)
        if chernoff == 0.0
            halflife = Inf
        else
            halflife = log(2.0) / chernoff
        end
        store_matrix2[row_index, 12:13] = [chernoff, halflife];
        q, re, c, hl = worrisome_entropy(alpha_z_tilde, baseline.kappa_hat,
                                        baseline.alpha_k_hat, beta_tilde, baseline);
        store_matrix2[row_index, 14:17] = [q, re, c, hl]
        store_matrix2[row_index, 18:19] = H0'
        store_matrix2[row_index, 20:21] = H1'
        store_matrix2[row_index, 22] = l_star
        x0, x1, x2 = tilting_function(Worrisome(alpha_z_tilde,
                                                baseline.kappa_hat,
                                                baseline.alpha_k_hat,
                                                beta_tilde), baseline)
        store_matrix2[row_index, 23:25] = [x0, x1, x2]

    end
end
npzwrite("./data/model_singlecapital.npz", vcat(store_matrix, store_matrix2))